In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
from pathlib import Path

###  !rm -rf <folder_name>

data_path = Path("data/")
image_path = data_path

with zipfile.ZipFile("drive/MyDrive/emp/case1.zip", "r") as zip_ref:
  print("Unzipping case1...")
  zip_ref.extractall(image_path)


Unzipping case1...


In [ ]:
# # For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
# try:
#     import torch
#     import torchvision
#     assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
#     assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")
# except:
#     print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
#     !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#     import torch
#     import torchvision
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")

In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4393, done.
remote: Total 4393 (delta 0), reused 0 (delta 0), pack-reused 4393 (from 1)
Receiving objects: 100% (4393/4393), 764.14 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (2656/2656), done.
Updating files: 100% (248/248), done.


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Setup Dirs
# train_dir = image_path / "emp/case/train"
# test_dir = image_path / "emp/case/test"

# train_dir , test_dir

In [ ]:
# Setup dirs
train_dir = image_path / "case1/train"
test_dir = image_path / "case1/test"

# # Setup pretrained weights (plenty of these available in torchvision.models)
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# # Get transforms from weights (these are the transforms that were used to obtain the weights)
# automatic_transforms = weights.transforms()
# print(f"Automatically created transforms: {automatic_transforms}")

# # Create data loaders
# train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
#     train_dir=train_dir,
#     test_dir=test_dir,
#     transform=automatic_transforms, # use automatic created transforms
#     batch_size=32
# )

# train_dataloader, test_dataloader, class_names

In [ ]:
# try:
#     from torch.utils.tensorboard import SummaryWriter
# except:
#     print("[INFO] Couldn't find tensorboard... installing it.")
#     !pip install -q tensorboard
#     from torch.utils.tensorboard import SummaryWriter


# # Create a writer with all default settings
# writer = SummaryWriter()

In [ ]:
# def create_writer(experiment_name: str,
#                   model_name: str,
#                   extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
#     """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

#     log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

#     Where timestamp is the current date in YYYY-MM-DD format.

#     Args:
#         experiment_name (str): Name of experiment.
#         model_name (str): Name of model.
#         extra (str, optional): Anything extra to add to the directory. Defaults to None.

#     Returns:
#         torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

#     Example usage:
#         # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
#         writer = create_writer(experiment_name="data_10_percent",
#                                model_name="effnetb2",
#                                extra="5_epochs")
#         # The above is the same as:
#         writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
#     """
#     from datetime import datetime
#     import os

#     # Get timestamp of current date (all experiments on certain day live in same folder)
#     timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

#     if extra:
#         # Create log directory path
#         log_dir = os.path.join("runs", timestamp, str(experiment_name), model_name, extra)
#     else:
#         log_dir = os.path.join("runs", timestamp, str(experiment_name), model_name)

#     print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
#     return SummaryWriter(log_dir=log_dir)

In [ ]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Import train() function from:
# https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/going_modular/engine.py
from typing import Dict, List
from tqdm.auto import tqdm
import wandb


# Add writer parameter to train()
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          # writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        # if writer:
        #     # Add results to SummaryWriter
        #     writer.add_scalars(main_tag="Loss",
        #                        tag_scalar_dict={"train_loss": train_loss,
        #                                         "test_loss": test_loss},
        #                        global_step=epoch)
        #     writer.add_scalars(main_tag="Accuracy",
        #                        tag_scalar_dict={"train_acc": train_acc,
        #                                         "test_acc": test_acc},
        #                        global_step=epoch)

        #     # Close the writer
        #     writer.close()
        # else:
        #     pass
        ### End new ###

        # Log metrics to wandb
        wandb.log({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
            "epoch": epoch + 1
        })


    # Return the filled results at the end of the epochs
    return results

In [ ]:
# Setup pretrained weights (plenty of these available in torchvision.models)
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT


# Get transforms from weights (these are the transforms that were used to obtain the weights)
vit_transforms = pretrained_vit_weights.transforms()
print(f"Automatically created transforms: {vit_transforms}")

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [ ]:
BATCH_SIZE = 32

# Create vit training and test DataLoaders
train_dataloader_vit, test_dataloader_vit, class_names = data_setup.create_dataloaders(train_dir=train_dir,
    test_dir=test_dir,
    transform=vit_transforms,
    batch_size=BATCH_SIZE)
# Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)

In [ ]:
import torchvision
from torch import nn

# Get num out features (one for each class pizza, steak, sushi)
OUT_FEATURES = len(class_names)

# vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT


#6. Create an vit feature extractor
def create_vit():
    # 1. Get the base model with pretrained weights and send to target device
    vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    # 2. Setup a ViT model instance with pretrained weights
    model = torchvision.models.vit_b_16(weights=vit_weights).to(device)

    # 3. Freeze the base parameters
    for parameter in model.parameters():
        parameter.requires_grad = False

    # 4. Change the classifier head (set the seeds to ensure same initialization with linear head)
    model.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)

    # 5. Give the model a name
    model.name = "vit16"
    print(f"[INFO] Created new {model.name} model.")
    return model



In [ ]:
vit16 = create_vit()

# Print a summary using torchinfo (uncomment for actual output)
summary(model=vit16,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new vit16 model.


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 2]              768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [ ]:
# 1. Create epochs list
num_epochs = 50

# 2. Create models list (need to create a new model for each experiment)
models = ["vit16"]

# 3. Create dataloaders dictionary for various dataloaders
# train_dataloaders = {"data_train_effnet": train_dataloader_effnet,
#                      "data_train_inception": train_dataloader_inception,
#                      "data_train_mobilenet": train_dataloader_mobilenet,
#                      "data_train_resnet50": train_dataloader_resnet50,
#                      "data_train_vgg16": train_dataloader_vgg16}

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ellesar (ellesar-srbiau) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
%%time
from going_modular.going_modular.utils import save_model
import wandb


# 1. Set the random seeds
# set_seeds(seed=42)

# 2. Keep track of experiment numbers
experiment_number = 0

for model_name in models:

    # 6. Create information print outs
    experiment_number += 1
    print(f"[INFO] Experiment number: {experiment_number}")
    print(f"[INFO] Model: {model_name}")
    #print(f"[INFO] DataLoader: {dataloader_name}")
    print(f"[INFO] Number of epochs: {num_epochs}")

    # 7. Select the model

    if model_name == "vit16":
        model = create_vit() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_vit
        test_dataloader=test_dataloader_vit


    # 8. Create a new loss and optimizer for every model
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


    # Initialize a new wandb run for each experiment
    wandb.init(project="your-project-name",
               name=f"{model_name}_{num_epochs}_epochs_exp_{experiment_number}",
               config={
                   "model_name": model_name,
                   "num_epochs": num_epochs,
                   "batch_size": BATCH_SIZE,
                   "learning_rate": 0.001,
                   "loss_fn": loss_fn.__class__.__name__,
                   "optimizer": optimizer.__class__.__name__
               })


    # 9. Train target model with target dataloaders and track experiments
    train(model=model,
          train_dataloader=train_dataloader,
          test_dataloader=test_dataloader,
          optimizer=optimizer,
          loss_fn=loss_fn,
          epochs=num_epochs,
          device=device,
        #   writer=create_writer(experiment_name=experiment_number,
        #                         model_name=model_name,
        #                         extra=f"{num_epochs}_epochs")
          )

    # Finish the wandb run
    wandb.finish()

    # 10. Save the model to file so we can get back the best model
    save_filepath = f"01_{model_name}_{num_epochs}_epochs.pth"
    save_model(model=model,
                target_dir="models",
                model_name=save_filepath)
    print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: vit16
[INFO] Number of epochs: 50
[INFO] Created new vit16 model.


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5386 | train_acc: 0.7363 | test_loss: 0.3679 | test_acc: 0.8672
Epoch: 2 | train_loss: 0.2986 | train_acc: 0.9056 | test_loss: 0.2319 | test_acc: 0.9609
Epoch: 3 | train_loss: 0.2097 | train_acc: 0.9395 | test_loss: 0.1868 | test_acc: 0.9688
Epoch: 4 | train_loss: 0.1649 | train_acc: 0.9531 | test_loss: 0.1817 | test_acc: 0.9297
Epoch: 5 | train_loss: 0.1439 | train_acc: 0.9570 | test_loss: 0.1307 | test_acc: 0.9766
Epoch: 6 | train_loss: 0.1232 | train_acc: 0.9746 | test_loss: 0.1251 | test_acc: 0.9844
Epoch: 7 | train_loss: 0.1073 | train_acc: 0.9844 | test_loss: 0.1151 | test_acc: 0.9844
Epoch: 8 | train_loss: 0.0974 | train_acc: 0.9863 | test_loss: 0.1024 | test_acc: 0.9766
Epoch: 9 | train_loss: 0.0840 | train_acc: 0.9941 | test_loss: 0.0984 | test_acc: 0.9844
Epoch: 10 | train_loss: 0.0839 | train_acc: 0.9844 | test_loss: 0.0856 | test_acc: 0.9844
Epoch: 11 | train_loss: 0.0739 | train_acc: 0.9922 | test_loss: 0.0805 | test_acc: 0.9844
Epoch: 12 | train_l

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_acc,▁▆▆▄▇▇▇▇▇▇██████████████████████████████
test_loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▇▇███████████████████████████████████
train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
test_acc,1
test_loss,0.02395
train_acc,1
train_loss,0.01216


[INFO] Saving model to: models/01_vit16_50_epochs.pth
--------------------------------------------------

CPU times: user 4min 52s, sys: 13.8 s, total: 5min 6s
Wall time: 7min 40s


In [ ]:
# # Viewing TensorBoard in Jupyter and Google Colab Notebooks (uncomment to view full TensorBoard instance)
# %load_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# %cp -av models drive/MyDrive/emp/

'models/01_vit16_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_vit16_50_epochs.pth'


In [ ]:
# # Setup the best model filepath
# best_model_path = "models/07_effnetb2_data_20_percent_10_epochs.pth"

# # Instantiate a new instance of EffNetB2 (to load the saved state_dict() to)
# best_model = create_effnetb2()

# # Load the saved best model state_dict()
# best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
# # Check the model file size
# from pathlib import Path

# # Get the model size in bytes then convert to megabytes
# effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
# print(f"EfficientNetB2 feature extractor model size: {effnetb2_model_size} MB")

In [ ]:
# # Import function to make predictions on images and plot them
# # See the function previously created in section: https://www.learnpytorch.io/06_pytorch_transfer_learning/#6-make-predictions-on-images-from-the-test-set
# from going_modular.going_modular.predictions import pred_and_plot_image

# # Get a random list of 3 images from 20% test set
import random
num_images_to_plot = 3
test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg")) # get all test image paths from 20% dataset
test_image_path_sample = random.sample(population=test_image_path_list,
                                       k=num_images_to_plot) # randomly select k number of images

# Iterate through random test image paths, make predictions on them and plot them
for image_path in test_image_path_sample:
    pred_and_plot_image(model=best_model,
                        image_path=image_path,
                        class_names=class_names,
                        image_size=(224, 224))

NameError: name 'data_20_percent_path' is not defined

In [ ]:
# Predict on custom image
pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names)

NameError: name 'pred_and_plot_image' is not defined